In [1]:
from tkinter import *
from tkinter import filedialog
import cv2
from PIL import Image,ImageTk
import numpy as np
import matplotlib.pyplot as plt
from tkinter import messagebox
from tkinter import ttk
import os
from PIL import Image as PILImage
root=Tk()
root.geometry('1245x700')
root.config(bg='#1c2833')
root.title('TB CLASSIFIER')
title=Label(root,text='TB CLASSIFIER',height=2,bg='cyan4')
title.pack(side=TOP,fill=X)

#initailize the variables
filepath=''
selected_image=None
#processed_image=None
def close():
    user_response = messagebox.askyesno("Exit", "Are you sure you want to exit?")

    if user_response:
        root.destroy()
from tkinter import filedialog

def load_image():
    global filepath, selected_image

    # Open the file dialog to choose an image file
    filepath = filedialog.askopenfilename(filetypes=(('PNG files', '*.png'), ('JPEG files', '*.jpg')))

    if filepath:
        # Load the image using OpenCV
        img = cv2.imread(filepath)

        # Convert BGR image to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Convert the image to a PIL Image
        selected_pil = Image.fromarray(img_rgb)

        # Create an ImageTk object for displaying on the canvas
        selected_image = ImageTk.PhotoImage(selected_pil)

        # Display the image on the canvas
        selected_image_canvas.create_image(0, 0, anchor='nw', image=selected_image)
        selected_image_canvas.image = selected_image

        # Print the filepath to verify
        print(f"Selected File: {filepath}")
    else:
        print("No file selected.")

def reset():
    selected_image_canvas.delete('all')

def visualize_image():
   global filepath
   if not filepath:
        print("Please load an image first.")
        return
   img = cv2.imread(filepath)
   # Convert the image to grayscale
   gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

   #convert image to
   hgr=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)


   visualize_image_tk = ImageTk.PhotoImage(Image.fromarray(gray_img))
   selected_image_canvas.create_image(0, 0, anchor='nw', image=visualize_image_tk)
   selected_image_canvas.image = visualize_image_tk

   create_zoom_panel(gray_img)

def create_zoom_panel(image):
    zoom_factor = 1.5  # Adjust this value for the initial zoom level
    zoom_panel = Toplevel(root)
    zoom_panel.title("Zoom Panel")

    # Function to handle mouse wheel event for zooming
    def on_mouse_wheel(event):
        nonlocal zoom_factor
        if event.delta > 0:
            zoom_factor *= 1.1
        else:
            zoom_factor /= 1.1
        update_zoomed_image()

    zoom_panel.bind("<MouseWheel>", on_mouse_wheel)

    zoomed_image_label = Label(zoom_panel)
    zoomed_image_label.pack()

    def update_zoomed_image():
        nonlocal zoom_factor
        zoomed_img = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor, interpolation=cv2.INTER_LINEAR)
        zoomed_image_tk = ImageTk.PhotoImage(Image.fromarray(zoomed_img))
        zoomed_image_label.configure(image=zoomed_image_tk)
        zoomed_image_label.image = zoomed_image_tk

    update_zoomed_image()

def enhance_image():
    global filepath

    if not filepath:
        print("Please load an image first.")
        return

    # Read the image using OpenCV
    img = cv2.imread(filepath)

    # Convert the BGR image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply median filtering
    median_filtered_img = cv2.medianBlur(gray_img, 5)  # Adjust the kernel size (5 in this case)

    # Apply histogram equalization
    enhanced_img = cv2.equalizeHist(median_filtered_img)

    # Update the displayed image on the canvas
    enhanced_image_tk = ImageTk.PhotoImage(Image.fromarray(enhanced_img))
    selected_image_canvas.create_image(0, 0, anchor='nw', image=enhanced_image_tk)
    selected_image_canvas.image = enhanced_image_tk

    # Plot the original and enhanced histograms
    plot_histogram(gray_img, "Original Histogram")
    plot_histogram(enhanced_img, "Enhanced Histogram")

def plot_histogram(image, title):
    plt.figure()
    plt.hist(image.ravel(), bins=256, range=[0, 256], density=True, color='gray', alpha=0.75)
    plt.title(title)
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.show()

def segment_image():
    global filepath, selected_image

    if not filepath:
        print("Please load an image first.")
        return

    # Reading the image using OpenCV
    img = cv2.imread(filepath)

    # Converting the BGR image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, segmented_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY)

    # Updating the displayed image on the canvas
    segmented_image_tk = ImageTk.PhotoImage(Image.fromarray(segmented_img))
    selected_image_canvas.create_image(0, 0, anchor='nw', image=segmented_image_tk)
    selected_image_canvas.image = segmented_image_tk


def extract_features():
    global filepath, selected_image

    if not filepath:
        print("Please load an image first.")
        return

    # Read the image using OpenCV
    img = cv2.imread(filepath)

    # Convert the BGR image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gabor filter for texture extraction
    gabor_kernel = cv2.getGaborKernel((21, 21), 5.0, np.pi / 4, 10.0, 1.0, 0, ktype=cv2.CV_32F)
    texture_img = cv2.filter2D(gray_img, cv2.CV_8UC3, gabor_kernel)

    # Calculate the color histogram for the original image
    hist_original = cv2.calcHist([img], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    hist_original = hist_original / hist_original.sum()  # Normalize the histogram

    # Calculate the histogram for the texture image
    hist_texture = cv2.calcHist([texture_img], [0], None, [256], [0, 256])
    hist_texture = hist_texture / hist_texture.sum()  # Normalize the histogram

    # Update the displayed images on the canvas
    texture_image_tk = ImageTk.PhotoImage(Image.fromarray(texture_img))
    selected_image_canvas.create_image(0, 0, anchor='nw', image=texture_image_tk)
    selected_image_canvas.image = texture_image_tk

    # Display histograms using matplotlib
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.plot(hist_original.flatten())  # Flatten the histogram array
    plt.title('Original Image Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    plt.plot(hist_texture.flatten())  # Flatten the histogram array
    plt.title('Texture Image Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

def classify_image():
   pass




#creating a buttonframe
techniframe=Frame(root,width=300,height=400,bg='#1c2833',bd=2,relief=RIDGE)
techniframe.place(x=530,y=45)

buttonframe=Frame(root,width=400,height=400,bg='#1c2833',bd=2,relief=RIDGE)
buttonframe.place(x=850,y=45)

#adding buttons in the buttonframe
action=Label(buttonframe,text='ACTION',bg='gray')
action.place(x=150,y=5)

show_button =Button(buttonframe, text="ZOOM IMAGE",width=30,bg='gray',command=visualize_image)
show_button.place(x=50,y=30)

enhance_buton =Button(buttonframe, text="Enhance Image",width=30,bg='gray',command=enhance_image)
enhance_buton.place(x=50,y=90)

segment_button =Button(buttonframe, text="Segment Image",width=30,bg='gray',command=segment_image)
segment_button.place(x=50,y=150)

features_button =Button(buttonframe, text="Feature Extraction",width=30,bg='gray',command=extract_features)
features_button.place(x=50,y=210)

classify_button =Button(buttonframe, text="Classification",width=30,bg='gray',command=classify_image)
classify_button.place(x=50,y=270)

#adding features in the technique frame
techniques=Label(techniframe,text='Technique',bg='#17202a',fg='white')
techniques.place(x=110,y=5)

enhe=Label(techniframe,text='Enhancement technique',bg='#17202a',fg='white')
enhe.place(x=50,y=50)

GusianBlur=ttk.Combobox(techniframe,values=['Meduim Filter','Histogram Equalization'])
GusianBlur.place(x=50,y=90)

seg=Label(techniframe,text='Segmentation',bg='#17202a',fg='white')
seg.place(x=50,y=120)

thresh=ttk.Combobox(techniframe,values=['THreshold'])
thresh.place(x=50,y=150)


feature=Label(techniframe,text='Feature Extraction',bg='#17202a',fg='white')
feature.place(x=50,y=180)

featurex=ttk.Combobox(techniframe,values=['Texture Analysis','HOG'])
featurex.place(x=50,y=210)

#creating acanvas frame
displayframe=Frame(root,width=500,height=400,bg='#17202a')
displayframe.place(x=10,y=45)

#OTHER BUTTONS
load_button =Button(root, text="Load Image",width=40,bg='blue',activebackground='blue',command=load_image)
load_button.place(x=40,y=470)

Reset_button =Button(root, text="Reset Screen",width=40,bg='green',activebackground='green',command=reset)
Reset_button.place(x=40,y=520)

#Creating results frame
Resultsframe=Frame(root,width=400,height=80,bg='gray',bd=2,relief=RIDGE)
Resultsframe.place(x=620,y=470)

# Label to display classification result
result_label = Label(Resultsframe, text='processing........', font=('Helvetica', 12), bg='gray')
result_label.place(x=10, y=10)


#creating a close button
closebutton=Button(root,text='CLOSE APPLICATION',width=150,bg='#2e4053',fg='black',activebackground='#2e4053',height=2,command=close)
closebutton.place(x=20,y=600)

selected_image_canvas=Canvas(displayframe,width=490,height=385,bg='black')
selected_image_canvas.place(x=5,y=5)

root.mainloop()

TclError: no display name and no $DISPLAY environment variable